# Gradient-Regularized Scalar Fluid: Linear Growth Notebook

This notebook is a self-contained testbench for the linear theory:

\[
\ddot{\delta}_k + 2H\dot{\delta}_k + \left(\frac{c_s^2(a)k^2}{a^2} + \frac{\kappa_{\rm eff}k^4}{a^4} - 4\pi G\bar\rho(a)\right)\delta_k = 0.
\]

It provides background functions, a configurable spinodal window for \(c_s^2(a)\), numerical integration of the growth equation in \(\ln a\), and basic visualizations for growth and stability bands.

For many tests it is convenient to set \(H_0=1\) and \(G=1\), making the system dimensionless.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def logspace(a, b, n=200):
    return np.exp(np.linspace(np.log(a), np.log(b), n))


## 1. Background cosmology \(H(a)\) and \(\bar\rho(a)\)

We model a flat \(\Lambda\)CDM background:
\[
H(a) = H_0 \sqrt{\Omega_r a^{-4} + \Omega_m a^{-3} + \Omega_\Lambda}.
\]
The clustered matter density is
\[
\bar\rho_m(a) = \rho_{m0}\,a^{-3},\qquad \rho_{m0} = \frac{3H_0^2}{8\pi G}\Omega_m.
\]


In [ ]:
# Background parameters (dimensionless defaults)
H0 = 1.0
G  = 1.0

Omega_r = 0.0
Omega_m = 1.0
Omega_L = 0.0

def H_of_a(a):
    return H0*np.sqrt(Omega_r*a**(-4) + Omega_m*a**(-3) + Omega_L)

def rho_m_of_a(a):
    rho_m0 = 3*H0**2*Omega_m/(8*np.pi*G)
    return rho_m0*a**(-3)

def four_pi_G_rho(a):
    return 4*np.pi*G*rho_m_of_a(a)


## 2. Model inputs: \(c_s^2(a)\) with a finite spinodal interval and \(\kappa_{\rm eff}\)

We implement \(c_s^2(a)\) as a baseline value plus a window where it is set negative. Replace this with any smooth function as needed.


In [ ]:
# Spinodal window for c_s^2(a)
a_on  = 0.20
a_off = 0.40

cs2_outside = 0.0
cs2_inside  = -1e-4

def cs2_of_a(a):
    a = np.asarray(a)
    cs2 = np.full_like(a, cs2_outside, dtype=float)
    mask = (a > a_on) & (a < a_off)
    cs2[mask] = cs2_inside
    return cs2

# Gradient regulator
kappa_eff = 1e-6


## 3. Growth equation in \(x=\ln a\)

Using \(d/dt = H(a)\,d/dx\), the growth equation becomes
\[
\delta'' + \left(2 + \frac{d\ln H}{dx}\right)\delta' + \frac{1}{H^2}\left(\frac{c_s^2 k^2}{a^2} + \frac{\kappa_{\rm eff}k^4}{a^4} - 4\pi G\bar\rho\right)\delta = 0,
\]
where primes denote \(d/dx\).


In [ ]:
def dlnH_dx(a, eps=1e-6):
    x = np.log(a)
    a1 = np.exp(x - eps)
    a2 = np.exp(x + eps)
    return (np.log(H_of_a(a2)) - np.log(H_of_a(a1))) / (2*eps)

def omega2_term(a, k):
    return cs2_of_a(a)*(k**2)/(a**2) + kappa_eff*(k**4)/(a**4) - four_pi_G_rho(a)

def Q_of_a_k(a, k):
    return omega2_term(a, k)/(H_of_a(a)**2)

def integrate_growth_ln_a(k, a_i=1e-3, a_f=1.0, n_steps=800, delta_i=None, delta_prime_i=None):
    x_i = np.log(a_i)
    x_f = np.log(a_f)
    xs = np.linspace(x_i, x_f, n_steps)
    dx = xs[1]-xs[0]
    a_grid = np.exp(xs)

    # Growing-mode-like IC in EdS: delta ~ a => delta' = delta in x=ln a
    if delta_i is None:
        delta_i = a_i
    if delta_prime_i is None:
        delta_prime_i = delta_i

    y = np.zeros((n_steps, 2), dtype=float)
    y[0,0] = delta_i
    y[0,1] = delta_prime_i

    def f(x, yvec):
        a = np.exp(x)
        d, dp = yvec
        A = 2.0 + dlnH_dx(a)
        Q = Q_of_a_k(a, k)
        return np.array([dp, -A*dp - Q*d], dtype=float)

    for i in range(n_steps-1):
        x = xs[i]
        yi = y[i]
        k1 = f(x, yi)
        k2 = f(x + 0.5*dx, yi + 0.5*dx*k1)
        k3 = f(x + 0.5*dx, yi + 0.5*dx*k2)
        k4 = f(x + dx, yi + dx*k3)
        y[i+1] = yi + (dx/6.0)*(k1 + 2*k2 + 2*k3 + k4)

    return a_grid, y[:,0], y[:,1]


## 4. Sanity check: EdS dust limit

Set \(\Omega_m=1\), \(\Omega_\Lambda=0\), \(c_s^2=0\), \(\kappa_{\rm eff}=0\). The growing mode should satisfy \(D(a)\propto a\).


In [ ]:
def run_sanity_test():
    global kappa_eff, cs2_inside, cs2_outside, Omega_m, Omega_L, Omega_r
    old = (kappa_eff, cs2_inside, cs2_outside, Omega_m, Omega_L, Omega_r)

    Omega_m, Omega_L, Omega_r = 1.0, 0.0, 0.0
    kappa_eff = 0.0
    cs2_outside = 0.0
    cs2_inside = 0.0

    k = 0.1
    a, d, dp = integrate_growth_ln_a(k, a_i=1e-3, a_f=1.0, n_steps=700)
    D = d/d[0]
    D_expected = a/a[0]

    plt.figure()
    plt.plot(a, D, label="Numerical D(a)")
    plt.plot(a, D_expected, label="a/a_i")
    plt.xscale("log")
    plt.yscale("log")
    plt.xlabel("a")
    plt.ylabel("D(a)")
    plt.title("Sanity check: EdS dust (expect D ~ a)")
    plt.legend()
    plt.show()

    (kappa_eff, cs2_inside, cs2_outside, Omega_m, Omega_L, Omega_r) = old

run_sanity_test()


## 5. Dispersion and stability band diagnostics

For fixed \(a\), the instantaneous dispersion structure is
\[
\omega^2(k,a) = \frac{c_s^2(a)k^2}{a^2} + \frac{\kappa_{\rm eff}k^4}{a^4} - 4\pi G\bar\rho(a).
\]
This section visualizes \(\omega^2\) and highlights unstable regions where \(\omega^2<0\).


In [ ]:
def omega2(a, k):
    return omega2_term(a, k)

def plot_dispersion(a=0.3, kmin=1e-3, kmax=1e2, n=500):
    ks = logspace(kmin, kmax, n)
    w2 = omega2(a, ks)

    plt.figure()
    plt.plot(ks, w2)
    plt.xscale("log")
    plt.xlabel("comoving k")
    plt.ylabel(r"$\omega^2(k,a)$")
    plt.title(f"Instantaneous dispersion at a={a}")
    plt.axhline(0.0)
    plt.show()

plot_dispersion(a=0.3)


## 6. Integrate growth for a grid of \(k\) and visualize \(D(k,a)\)

We integrate the growth equation for multiple wavenumbers and plot the normalized growth \(D(k,a)=\delta_k(a)/\delta_k(a_i)\).


In [ ]:
def run_grid(ks, a_i=1e-3, a_f=1.0, n_steps=900):
    out = {}
    for k in ks:
        a, d, dp = integrate_growth_ln_a(k, a_i=a_i, a_f=a_f, n_steps=n_steps)
        out[k] = (a, d, dp)
    return out

ks = logspace(1e-2, 1e1, 10)
grid = run_grid(ks)

plt.figure()
for k, (a, d, dp) in grid.items():
    D = d/d[0]
    plt.plot(a, D, label=f"k={k:.2g}")
plt.xscale("log")
plt.yscale("log")
plt.xlabel("a")
plt.ylabel(r"$D(k,a)$")
plt.title("Scale-dependent growth D(k,a)")
plt.legend(ncol=2, fontsize=8)
plt.show()


## 7. Transfer-like ratio relative to EdS dust growth

Define
\[
R(k) \equiv \frac{\delta_k(a_0)/\delta_k(a_i)}{a_0/a_i},
\]
which equals unity for EdS dust. This isolates late-time scale dependence from the regulated sector.


In [ ]:
def transfer_like_ratio(k, a_i=1e-3, a_0=1.0, n_steps=900):
    a, d, dp = integrate_growth_ln_a(k, a_i=a_i, a_f=a_0, n_steps=n_steps)
    D = d[-1]/d[0]
    D_std = a_0/a_i
    return D/D_std

ratios = np.array([transfer_like_ratio(k) for k in ks])

plt.figure()
plt.plot(ks, ratios)
plt.xscale("log")
plt.xlabel("k")
plt.ylabel("R(k)")
plt.title("Transfer-like ratio relative to EdS dust growth")
plt.axhline(1.0)
plt.show()


## 8. Parameter sweeps

Sweep over \(\kappa_{\rm eff}\) (and optionally the spinodal magnitude) and compare the resulting \(R(k)\).


In [ ]:
def sweep_kappa(kappas, ks, cs2_in=None):
    global kappa_eff, cs2_inside
    old_kappa = kappa_eff
    old_cs2in = cs2_inside
    if cs2_in is not None:
        cs2_inside = cs2_in

    results = {}
    for kap in kappas:
        kappa_eff = kap
        results[kap] = np.array([transfer_like_ratio(k) for k in ks])

    kappa_eff = old_kappa
    cs2_inside = old_cs2in
    return results

kappas = [1e-8, 1e-7, 1e-6, 1e-5]
sres = sweep_kappa(kappas, ks)

plt.figure()
for kap, rr in sres.items():
    plt.plot(ks, rr, label=f"kappa={kap:g}")
plt.xscale("log")
plt.xlabel("k")
plt.ylabel("R(k)")
plt.title("Effect of kappa_eff on scale-dependent growth")
plt.axhline(1.0)
plt.legend()
plt.show()


## 9. Notes for extension

To go beyond this notebook, you can replace the boxcar spinodal window with a smooth model derived from an explicit \(\varepsilon(n)\) and \(p(n)\), match initial conditions through radiation--matter equality, or couple the perturbations to a Boltzmann solver for a full transfer function.
